In [1]:
import numpy as np
import multiprocessing as mp

from functools import partial
from matplotlib import pyplot as plt

In [15]:
"""
Gera uma reta aleatória
"""
def f(x):
    
    y = np.square(x[:, 0]) + np.square(x[:, 1]) - 0.6
    
    y = np.where(y > 0, 1, -1)

    return y

def add_noise(y, perc_noise=0.1):
    n_noise = int(y.shape[0] * perc_noise)
    
    indices = np.arange(y.shape[0])
    
    np.random.shuffle(indices)
    
    noise = indices[:n_noise]
    
    y[noise] = -y[noise]
    
    return y

"""
Gera os dados baseados em uma reta
"""
def generate_data(n):
    
    # Gera 'n' pontos (x, y) aleatoriamente entre 0 e 1
    # e os converte para o intervalo -1 e 1
    points = (np.random.rand(n, 2) * 2) - 1
    
    return points

"""
Calcula os valores de saída da função ideal
"""
def calc_linear_y(weights, data):
    
    y = weights[0] + np.dot(data, weights[1:])
    
    # Transforma os valores em -1 ou 1
    # Valores em cima da reta são considerados -1
    y = np.where(y > 0, 1, -1)
    
    return y

In [3]:
def lin_regression(x, y):
    
    m_x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)
    
    x_dagger = np.dot( np.linalg.inv( np.dot(m_x.T , m_x) ), m_x.T)

    w_lin = np.dot(x_dagger, y)
    
    return w_lin

In [20]:
def calc_error(line, w):
    
    # Gera mil pontos para serem avaliados
    ev_data = generate_data(line, 1000)
    
    # Saída ideal
    ev_f_y = calc_y(line, ev_data)
    
    # Saída gerada pela g(x)
    ev_g_y = calc_y(w, ev_data)
    
    # Conta todos os pontos em que as saídas não foram iguais
    misclassified = np.count_nonzero(ev_f_y - ev_g_y)

    # Calcula a porcentagem dos pontos classificados erroneamente
    return misclassified / ev_data.shape[0]


def experiment_1(N):
  
    np.random.seed()
    
#     line = generate_line()

    x = generate_data(N)
    
    y = f(x)
    
    y = add_noise(y, 0.1)
    
    w_lin = lin_regression(x, y)
    
    y_g = calc_linear_y(w_lin, x)

    e_in = np.count_nonzero( y - y_g ) / y.shape[0]
#     e_out = calc_error(line, w_lin)
    
#     w_perc, it = perceptron(x, y, weights=w_lin)
    
    return e_in
#     return [e_in, e_out, it]
#     np.random.seed()
    
#     line = generate_line()

#     x = generate_data(line, N)
#     y = calc_y(line, x)
    
#     w_lin = lin_regression(x, y)
    
#     y_g = calc_y(w_lin, x)

#     e_in = np.count_nonzero( y - y_g ) / y.shape[0]
#     e_out = calc_error(line, w_lin)
    
#     w_perc, it = perceptron(x, y, weights=w_lin)
    
#     return [e_in, e_out, it]

In [5]:
"""
Calcula os valores de saída da função ideal
"""
def calc_non_linear_y(weights, data):
    
#     y = weights[0] + np.dot(data, weights[1:])
    y = weights[0] + np.dot(data, weights[1:])
    
    # Transforma os valores em -1 ou 1
    # Valores em cima da reta são considerados -1
    y = np.where(y > 0, 1, -1)
    
    return y

def experiment_2(N):
  
    np.random.seed()
    
#     line = generate_line()

    x = generate_data(N)
    y = f(x)
    
    w_lin = lin_regression(x, y)
    
    y_g = calc_linear_y(w_lin, x)

    e_in = np.count_nonzero( y - y_g ) / y.shape[0]
    
    return e_in

In [8]:
"""
Executa um certo número de experimentos paralelamente
Caso o número de processos não seja espeficidado,
o multiprocessing utiliza o valor padrão,
que costuma ser o número de processadores
"""
def run_experiment(N, num_exp, exp_id, processes=None):
    pool = mp.Pool(processes)
    
    function = None
    
    if exp_id == 1:
        function = experiment_1
    elif exp_id == 2:
        function = experiment_2
    else:
        print('Invalid experiment!')
        return None
    # Executa os experimentos 'num_exp' vezes, passando como
    # parâmetro para cada um, o número de dados N a serem gerados
    results = np.array(pool.map(function, [N] * num_exp))
    
    pool.close()
    
    # Calcula a média dos resuldados por coluna (iterações, erro)
    return np.mean(results, axis=0)

In [22]:
run_experiment(1000, 1000, 1)

0.50347500000000001

In [ ]:
#run_experiment(1000, 1000)